In [19]:
import random
from typing import Dict, List
import MeCab
import re
import pandas as pd 
data1=pd.read_csv("/Users/withmocha/Desktop/DATA/Capston Design(2024)/data(sentence)/0521(final)/data/normal/일반민원.csv",index_col=0)


In [20]:
data1

,sentence,predict
0,이용하시는 은행의 사이트에서 지방세 납부가 가능합니다.,0
1,그럼 스마트폰에서 기업은행 어플을 설치하시면 납부가 가능합니다.,0
2,방문납부도 가능합니다.,0
3,도보로 10분위치에 서울역지점이 있습니다.,0
4,도보로 가시는게 빠를 것 같습니다.,0
...,...,...
20065,62400원 입니다.,0
20066,1423200원 입니다.,0
20067,"입주신청서와 추천서, 급여내역서 외 기타 서류를 갖고 홈페이지에서 신청하면 됩니다.",0
20068,1~3순위가 있습니다.,0


In [21]:
import random
from typing import Dict, List
import MeCab

# Mecab 형태소 분석기 초기화
mecab = MeCab.Tagger()

In [22]:
similar_words_dict = {
    '가능': ['할 수 있음', '실행 가능', '가능성 있음', '할 수 있는'],
    '신청': ['요청', '등록 신청', '가입 신청', '신청서 제출'],
    '일': ['날짜', '날', '사건', '업무'],
    '원': ['돈', '금액', '화폐 단위', '원화'],
    '월': ['달', '한 달', '월간', '개월'],
    '시': ['시간', '때', '시각', '시기'],
    '만': ['단지', '오직', '다만', '그냥'],
    '확인': ['검토', '인증', '점검', '확인하기'],
    '지원': ['후원', '도움', '보조', '지원하기'],
    '경우': ['상황', '사례', '경우의 수', '경우에 따라'],
    '서울': ['서울시', '서울특별시', '서울 지역', '서울시내'],
    '발급': ['발행', '지급', '발행하기', '발급받기'],
    '년': ['해', '연도', '한 해', '연간'],
    '말씀': ['말', '발언', '이야기', '언급'],
    '등': ['기타', '외', '등등', '그리고'],
    '이용': ['사용', '활용', '이용하기', '이용함'],
    '것': ['물건', '사물', '사항', '일'],
    '후': ['나중에', '뒤에', '이후', '다음에'],
    '센터': ['기관', '조직', '센터 기관', '중심'],
    '감사': ['고마움', '감사함', '고맙습니다', '사례'],
    '신고': ['보고', '신고서 제출', '고발', '신고하기'],
    '홈페이지': ['웹사이트', '사이트', '온라인 페이지', '홈페이지 주소'],
    '납부': ['지불', '결제', '납입', '납부하기'],
    '방문': ['찾아가기', '내방', '방문하기', '방문함'],
    '무엇': ['어떤 것', '무슨', '무엇이든', '무엇인가'],
    '사용': ['이용', '활용', '사용하기', '사용함'],
    '문의': ['질문', '질의', '문의하기', '문의사항'],
    '서울시': ['서울', '서울특별시', '서울시내', '서울 지역'],
    '사항': ['내용', '사안', '항목', '주제'],
    '필요': ['요구', '필수', '필요성', '필요함'],
    '주민': ['거주자', '시민', '주민등록', '주민들'],
    '중': ['가운데', '중간', '도중', '중에서'],
    '등록': ['가입', '신청', '기록', '등록하기'],
    '분': ['사람', '개인', '분량', '부분'],
    '가요': ['노래', '음악', '가요곡', '음악곡'],
    '거': ['것', '물건', '사항', '사물'],
    '해당': ['관련', '해당됨', '관련됨', '해당 사항'],
    '지역': ['구역', '지방', '지역 사회', '구역내'],
    '이상': ['초과', '그 이상', '이상으로', '이상함'],
}






In [23]:
def augment_sentence(sentence: str, similar_words_dict: Dict[str, List[str]]) -> str:
    parsed = mecab.parse(sentence)
    words = [line.split('\t')[0] for line in parsed.split('\n') if line.strip() != 'EOS' and line.strip() != '']
    

    word_positions = []
    start = 0
    for word in words:
        start = sentence.find(word, start)
        word_positions.append((start, word))
        start += len(word)

    augmented_sentence = sentence


    for start, word in reversed(word_positions):
        found_similar = False

        for key in similar_words_dict.keys():
            if word == key:
               
                augmented_word = random.choice(similar_words_dict[key])
                augmented_sentence = augmented_sentence[:start] + augmented_word + augmented_sentence[start+len(word):]
                found_similar = True
                break
        
    return augmented_sentence

In [24]:
X1=pd.DataFrame(index=range(data1.shape[0]*2),columns=['sentence','predict'])

In [29]:
count=0
j=0
for i in range(data1.shape[0]*2):
    A=augment_sentence(data1.iloc[count,0],similar_words_dict)
    if A==data1.iloc[j,0]:
        continue
    else:
        X1.iloc[i,0]=A
        X1.iloc[i,1]=0
        count=count+1
        j=j+1
        
    if j>data1.shape[0]:
        j=0

In [30]:
X1

,sentence,predict
0,활용하시간는 은행의 사이트에서 지방세 납부하기가 실행 가능합니다.,0
1,그럼 스마트폰에서 기업은행 어플을 설치하때면 결제가 가능성 있음합니다.,0
2,방문하기결제도 실행 가능합니다.,0
3,도보로 10분량위치에 서울역지점이 있습니다.,0
4,도보로 가시는게 빠를 일 같습니다.,0
...,...,...
33861,NaN,NaN
33862,NaN,NaN
33863,NaN,NaN
33864,NaN,NaN


In [31]:
X1.isnull().sum()

sentence    21484
predict     21484
dtype: int64

In [32]:
X1=pd.DataFrame(X1.dropna(axis=0))

In [34]:
X1

,sentence,predict
0,활용하시간는 은행의 사이트에서 지방세 납부하기가 실행 가능합니다.,0
1,그럼 스마트폰에서 기업은행 어플을 설치하때면 결제가 가능성 있음합니다.,0
2,방문하기결제도 실행 가능합니다.,0
3,도보로 10분량위치에 서울역지점이 있습니다.,0
4,도보로 가시는게 빠를 일 같습니다.,0
...,...,...
16926,개월소득 실수령액이 150단지금액 이하여야합니다.,0
16927,서울 지역소재 직장근무로 제한하고있습니다.,0
16928,62400원화 입니다.,0
16929,1423200돈 입니다.,0


In [35]:
X1.sample(frac=1)

,sentence,predict
14541,그러면 우리시각의 주최로 하는 ㅇㅇ축제가 있습니다,0
10603,2한 해에서 3연도 매월 근로소득으로 저축하는 금액의 동일한 금액을 서울 예산과 시...,0
8587,"네. 소비쿠폰, 특별돌봄쿠폰 받으신 사람들도 긴급재난지원금을 추가로 받으실 수 있습...",0
8812,전화로는 가입 신청이 어렵습니다.,0
3105,하시각는게 도움이 되실겁니다.,0
...,...,...
1267,네. 요청하시각기 전에 협약은행에서 상담을 받으시길 바랍니다.,0
16523,9개월 18날 사용함사람부터 적용됩니다.,0
14681,아 그렇군요 연락 주셔서 감사함합니다.,0
8968,네 신고서 제출가능성 있음합니다. 다만 일정기간은 계도기간을 가지기 때문에 과태료 ...,0


In [36]:
X1.isnull().sum()

sentence    0
predict     0
dtype: int64

In [37]:
X1

,sentence,predict
0,활용하시간는 은행의 사이트에서 지방세 납부하기가 실행 가능합니다.,0
1,그럼 스마트폰에서 기업은행 어플을 설치하때면 결제가 가능성 있음합니다.,0
2,방문하기결제도 실행 가능합니다.,0
3,도보로 10분량위치에 서울역지점이 있습니다.,0
4,도보로 가시는게 빠를 일 같습니다.,0
...,...,...
16926,개월소득 실수령액이 150단지금액 이하여야합니다.,0
16927,서울 지역소재 직장근무로 제한하고있습니다.,0
16928,62400원화 입니다.,0
16929,1423200돈 입니다.,0


In [38]:
X1.to_csv("일반 민원(증강).csv")